# OSFI SCSE Climate & Market Risk Calculations

https://www.osfi-bsif.gc.ca/en/data-forms/reporting-returns/standardized-climate-scenario-exercise

---

## 1️/ Climate-Adjusted Probability of Default (PD) (Section 3.4.4)

### **Formula**

$\text{climatePD}_t = \frac{1}{1 + \exp\left(-\left(\logit(\text{PD}) + \text{climateAdd\text{-}on}_{t,0}\right)\right)}$


### **Interpretation**
- Adjusts the baseline PD by adding a climate-specific risk factor in log-odds space.
- This results in a higher probability of default for riskier exposures.

---

## 2️/ Baseline Market Value of Equity Exposures (Section 3.5.2)

### **Formula**

$\text{baselineMarketValue}_T = \text{EquityExposure}_{Q4\,2023} \times \text{baselineEquityIndex}_T$

### **Interpretation**
- Adjusts initial exposure values using a projected equity index level.
- Captures baseline growth or shrinkage in market value before any shocks.

---

## 3️/ Change in Market Value Due to Equity Shocks (Section 3.5.2)

### **Formula**

$\Delta \text{MarketValue}_T = \text{baselineMarketValue}_T \times \text{EquityShock}_T$


### **Interpretation**
- Measures gains or losses due to market volatility.
- A negative shock results in a decline in market value.

---

## 4️/ Climate-Adjusted Credit Spread for Bonds (Section 3.5.3)

### **Steps**
1. **Calculate Climate-Adjusted PD** (from Section 3.4.4)
2. **Assign Credit Rating** based on adjusted PD:
   - PD < 0.01 → AAA  
   - PD < 0.02 → AA  
   - PD < 0.05 → A  
   - PD < 0.10 → BBB  
   - Else → BB or lower  
3. **Determine Credit Spread** based on assigned rating:
   - AAA → 0.50%  
   - AA → 1.00%  
   - A → 1.50%  
   - BBB → 2.00%  
   - BB or lower → 3.00%

### **Interpretation**
- Higher climate-adjusted PD results in a worse credit rating


In [ ]:
import pandas as pd
import numpy as np

# Sample loan data with baseline PD and climate adjustment
loan_data = pd.DataFrame({
    "Loan_ID": ["L001", "L002", "L003"],
    "Baseline_PD": [0.02, 0.04, 0.06],  # 2%, 4%, 6% probability of default
    "Climate_Add_On": [0.05, 0.02, 0.07]  # Climate risk adjustments
})

def logit(p):
    return np.log(p / (1 - p))

def climate_adjusted_pd(baseline_pd, climate_add_on):
    return 1 / (1 + np.exp(-(logit(baseline_pd) + climate_add_on)))

# Apply function to each row
loan_data["Climate_Adjusted_PD"] = loan_data.apply(
    lambda row: climate_adjusted_pd(row["Baseline_PD"], row["Climate_Add_On"]), axis=1
)

print(loan_data)


  Loan_ID  Baseline_PD  Climate_Add_On  Climate_Adjusted_PD
0    L001         0.02            0.05             0.021004
1    L002         0.04            0.02             0.040775
2    L003         0.06            0.07             0.064072


In [ ]:
# Sample equity exposures dataset
equity_data = pd.DataFrame({
    "Stock_ID": ["E001", "E002", "E003"],
    "Equity_Exposure_2023": [1_000_000, 750_000, 500_000],  # $1M, $750K, $500K
    "Baseline_Equity_Index_T": [1.05, 1.10, 0.95]  # Index growth/shrinkage
})

# Compute baseline market value
equity_data["Baseline_Market_Value_T"] = equity_data["Equity_Exposure_2023"] * equity_data["Baseline_Equity_Index_T"]

print(equity_data)

  Stock_ID  Equity_Exposure_2023  Baseline_Equity_Index_T  \
0     E001               1000000                     1.05   
1     E002                750000                     1.10   
2     E003                500000                     0.95   

   Baseline_Market_Value_T  
0                1050000.0  
1                 825000.0  
2                 475000.0  


In [ ]:
# Adding equity shock data
equity_data["Equity_Shock_T"] = [-0.10, 0.05, -0.20]  # -10%, +5%, -20%

# Compute change in market value
equity_data["Market_Value_Change"] = equity_data["Baseline_Market_Value_T"] * equity_data["Equity_Shock_T"]

print(equity_data)

  Stock_ID  Equity_Exposure_2023  Baseline_Equity_Index_T  \
0     E001               1000000                     1.05   
1     E002                750000                     1.10   
2     E003                500000                     0.95   

   Baseline_Market_Value_T  Equity_Shock_T  Market_Value_Change  
0                1050000.0           -0.10            -105000.0  
1                 825000.0            0.05              41250.0  
2                 475000.0           -0.20             -95000.0  


In [ ]:
# Sample bond data
bond_data = pd.DataFrame({
    "Bond_ID": ["B001", "B002", "B003"],
    "Baseline_PD": [0.01, 0.03, 0.07],  # Default probabilities
    "Climate_Add_On": [0.02, 0.01, 0.05]  # Climate risk adjustments
})

# Compute climate-adjusted PD
bond_data["Climate_Adjusted_PD"] = bond_data.apply(
    lambda row: climate_adjusted_pd(row["Baseline_PD"], row["Climate_Add_On"]), axis=1
)

def assign_credit_rating(adjusted_pd):
    if adjusted_pd < 0.01:
        return 'AAA'
    elif adjusted_pd < 0.02:
        return 'AA'
    elif adjusted_pd < 0.05:
        return 'A'
    elif adjusted_pd < 0.10:
        return 'BBB'
    else:
        return 'BB or lower'

def determine_credit_spread(rating):
    spreads = {'AAA': 0.005, 'AA': 0.01, 'A': 0.015, 'BBB': 0.02, 'BB or lower': 0.03}
    return spreads.get(rating, 0.05)

# Assign credit ratings and spreads
bond_data["Credit_Rating"] = bond_data["Climate_Adjusted_PD"].apply(assign_credit_rating)
bond_data["Credit_Spread"] = bond_data["Credit_Rating"].apply(determine_credit_spread)

print(bond_data)

  Bond_ID  Baseline_PD  Climate_Add_On  Climate_Adjusted_PD Credit_Rating  \
0    B001         0.01            0.02             0.010200            AA   
1    B002         0.03            0.01             0.030292             A   
2    B003         0.07            0.05             0.073326           BBB   

   Credit_Spread  
0          0.010  
1          0.015  
2          0.020  
